In [4]:

from py import log
from xtquant import xttrader
from xtquant import xtdata
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
from xtquant import xtconstant
import pandas_market_calendars as mcal
import pandas as pd
import akshare as ak

d:\anaconda3\envs\python3.8\lib\site-packages\exchange_calendars\exchange_calendar.py:2345: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


In [9]:
xtdata.download_history_data("603305.SH", period='1d', start_time='20241029', end_time='20241031', incrementally = None)
x1dpdata = xtdata.get_local_data(field_list=[], stock_list=['603305.SH'], period='1d', start_time='20241029', end_time='20241031', count=-1, dividend_type='none', fill_data=True)

In [8]:
x1dpdata

{'603305.SH':                    time   open   high    low  close  volume       amount  \
 20241029  1730131200000  12.52  13.52  12.52   13.0  741628  968310738.0   
 
           settelementPrice  openInterest  preClose  suspendFlag  
 20241029               0.0            15     12.42            0  }